In [1]:
from collections import defaultdict


def load_data(path):
    with open(path) as f:
        data = f.read().splitlines()
    return data


def get_start_end_coordinates(grid):
    start = end = None
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'S':
                start = i, j
            if grid[i][j] == 'E':
                end = i, j
            if start and end:
                return start, end
    return start, end


def grid2graph(grid):
    graph = {}
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == '#':
                continue
            for orientation in ['h', 'v']:
                coordinates = i, j
                node = coordinates, orientation
                neighbors = get_neighbors(grid, node)
                graph[node] = neighbors
    return graph


def get_neighbors(grid, node):
    coordinates, orientation = node
    x, y = coordinates
    x_min, y_min, x_max, y_max = 1, 1, len(grid)-2, len(grid[0])-2
    neighbors = {}
    if orientation == 'h':
        neighbors.update({(coordinates, 'v'): 1_000})
    elif orientation == 'v':
        neighbors.update({(coordinates, 'h'): 1_000})
    if x >= x_min:
        x_n, y_n = x - 1, y
        if grid[x_n][y_n] != '#':
            if orientation == 'v':
                neighbors.update({((x_n, y_n), 'v'): 1})
    if x <= x_max:
        x_n, y_n = x + 1, y
        if grid[x_n][y_n] != '#':
            if orientation == 'v':
                neighbors.update({((x_n, y_n), 'v'): 1})
    if y >= y_min:
        x_n, y_n = x, y - 1
        if grid[x_n][y_n] != '#':
            if orientation == 'h':
                neighbors.update({((x_n, y_n), 'h'): 1})
    if y <= y_max:
        x_n, y_n = x, y + 1
        if grid[x_n][y_n] != '#':
            if orientation == 'h':
                neighbors.update({((x_n, y_n), 'h'): 1})
    return neighbors


def go_backwards(start, node, previous_neighbors, positions=set()):
    if node == start:
        return
    for previous_neighbor in previous_neighbors[node]:
        positions.add(previous_neighbor)
        go_backwards(start, previous_neighbor, previous_neighbors, positions)
    return positions


def solve(graph, start, end):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    unvisited_nodes = set(graph)
    previous_neighbors = defaultdict(set)
    while unvisited_nodes:
        if (end, 'h') not in unvisited_nodes and (end, 'v') not in unvisited_nodes:
            break
        min_node = None
        for node in unvisited_nodes:
            if min_node is None or distances[node] < distances[min_node]:
                min_node = node
        unvisited_nodes.remove(min_node)
        for neighbor, price in graph[min_node].items():
            distance = distances[min_node] + price
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_neighbors[neighbor] = {min_node}
            elif distance == distances[neighbor]:
                previous_neighbors[neighbor].add(min_node)
    shortest_path = float('inf')
    end_node = None
    for node, price in distances.items():
        coordinates, orientation = node
        if coordinates == end and price < shortest_path:
            shortest_path = price
            end_node = node
    positions = go_backwards(start, end_node, previous_neighbors)
    positions.add(end_node)
    tiles = len(set([i[0] for i in positions]))
    return shortest_path, tiles


grid = load_data('input.txt')
start, end = get_start_end_coordinates(grid)
graph = grid2graph(grid)
start_node = start, 'h'
solve(graph, start_node, end)

(83432, 467)